In [11]:
import matplotlib
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier

In [12]:
df = pd.read_csv('data/train_data.csv')

In [13]:
df.head()

,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,7291,4058200630,20141002T000000,3,1.75,2190,7021,1.0,0,2,...,1390,800,1953,0,98178,47.5033,-122.232,2180,7155,353000
1,14835,3613600150,20150105T000000,3,2.50,2370,6840,2.0,0,0,...,2370,0,1987,0,98119,47.6503,-122.366,1590,4400,300523
2,15880,7170200080,20140617T000000,2,1.00,1230,3800,1.0,0,0,...,1230,0,1928,0,98115,47.6797,-122.292,1610,3800,435000
3,8812,9542300530,20141124T000000,4,2.25,2510,9963,1.0,0,0,...,2200,310,1967,0,98005,47.5973,-122.177,3110,9963,800000
4,17220,3626039028,20140818T000000,3,1.00,1160,7491,1.0,0,0,...,1160,0,1917,0,98177,47.7024,-122.359,1800,2267,417500


In [14]:
x_train, x_test, y_train, y_test = train_test_split(
    df[[
        'bedrooms',
        'bathrooms',
        'sqft_living',
        'sqft_lot',
        'floors',
        'waterfront',
        'view',
        'condition',
        'grade',
        'sqft_above',
        'sqft_basement',
        'yr_built',
        'yr_renovated',
        'sqft_living15',
        'sqft_lot15',
    ]], 
    df['price'], 
    test_size=0.33
)

In [26]:
regr = linear_model.LinearRegression()
regr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [27]:
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [28]:
def rmse(predictions, actual):
    return np.sqrt(np.mean((predictions - actual)**2))

print('Linear Regression:', rmse(regr.predict(x_test), y_test))
print('Decision Tree', rmse(clf.predict(x_test), y_test))

Linear Regression: 202121.00983026106
Decision Tree 269051.5555046699


# Submission File Generation

In [16]:
dfTest = pd.read_csv('data/val_data.csv').rename({'Unnamed: 0': 'Unique_idx'}, axis='columns')

In [17]:
dfTest['price'] = regr.predict(dfTest[[
    'bedrooms',
    'bathrooms',
    'sqft_living',
    'sqft_lot',
    'floors',
    'waterfront',
    'view',
    'condition',
    'grade',
    'sqft_above',
    'sqft_basement',
    'yr_built',
    'yr_renovated',
    'sqft_living15',
    'sqft_lot15',
]])

In [18]:
dfTest[['Unique_idx', 'price']].to_csv('submission.csv', index=None, header=True)